<a href="https://colab.research.google.com/github/raghunanden/image-classifiction-framework/blob/master/dataset/colab_import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split
import numpy as np
import os

In [1]:
! git clone https://github.com/raghunanden/image-classifiction-framework.git

Cloning into 'image-classifiction-framework'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 2), reused 22 (delta 2), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [2]:
from google.colab import auth
auth.authenticate_user()
project_id = 'vocal-facet-272514'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

gs://raghukaggledata/


In [3]:
bucket_name = 'raghukaggledata'
!gsutil -m cp -r gs://{bucket_name}/aptos2019-blindness-detection.zip /content/image-classifiction-framework/dataset/

Copying gs://raghukaggledata/aptos2019-blindness-detection.zip...
/ [1/1 files][  9.5 GiB/  9.5 GiB] 100% Done  53.8 MiB/s ETA 00:00:00           
Operation completed over 1 objects/9.5 GiB.                                      


In [0]:
## read_data_csv
## split it into train test and validation
## organize the images accordingly
## build input pipeline using tf.data

In [0]:
%%bash
cd /content/image-classifiction-framework/dataset/
unzip -qq aptos2019-blindness-detection.zip

In [0]:
train_df = pd.read_csv("/content/image-classifiction-framework/dataset/train.csv")

In [29]:
train_df.head(1)

,id_code,diagnosis
0,000c1434d8d7,2


In [0]:
X_train, X_eval, y_train, y_eval = train_test_split(
    train_df["id_code"], train_df["diagnosis"], test_size=0.25, random_state=42)
train = pd.DataFrame()
train["id_code"] = X_train
train["diagnosis"] = y_train
validation = pd.DataFrame()
validation["id_code"] = X_eval
validation["diagnosis"] = y_eval

In [0]:
def move_file(file_name, folder=None):
  # Source path

  source = '/content/image-classifiction-framework/dataset/train_images/{}.png'.format(file_name)
    
  # Destination path  
  destination = '/content/image-classifiction-framework/dataset/{}/{}.png'.format(folder,file_name)
    
  # Move the content of  
  # source to destination  
  dest = shutil.move(source, destination)  

In [39]:
if not os.path.exists('/content/image-classifiction-framework/dataset/train'):
    os.makedirs('/content/image-classifiction-framework/dataset/train')
if not os.path.exists('/content/image-classifiction-framework/dataset/validation'):
    os.makedirs('/content/image-classifiction-framework/dataset/validation')
train["id_code"].apply(lambda x: move_file(x, folder='train'))
validation["id_code"].apply(lambda x: move_file(x, folder='validation'))

2059    None
1105    None
818     None
1237    None
463     None
        ... 
1075    None
1108    None
2877    None
1798    None
2177    None
Name: id_code, Length: 916, dtype: object

In [0]:
train_image_path = "/content/image-classifiction-framework/dataset/train"
validation_image_path = "/content/image-classifiction-framework/dataset/validation"
test_image_path = "/content/image-classifiction-framework/dataset/validation"

In [0]:
import pathlib
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_data_dir = pathlib.Path(train_image_path)
validation_data_dir = pathlib.Path(validation_image_path)
test_data_dir = pathlib.Path(test_image_path)

In [0]:
train_ds = tf.data.Dataset.list_files(str(train_data_dir/"*"))
validation_ds = tf.data.Dataset.list_files(str(validation_data_dir/"*"))
test_ds = tf.data.Dataset.list_files(str(test_data_dir/"*"))

In [0]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
TRAIN_STEPS_PER_EPOCH = np.ceil(len(list(train_ds))/BATCH_SIZE)
VALIDATION_STEPS_PER_EPOCH = np.ceil(len(list(validation_ds))/BATCH_SIZE)
TEST_STEPS_PER_EPOCH = np.ceil(len(list(test_ds))/BATCH_SIZE)
CLASS_MAPPING_DICT = {0:"No DR",1:"Mild",2:"Moderate",3:"Severe",4:"Proliferative DR"}
CLASS_NAMES = np.array(list(CLASS_MAPPING_DICT.values()))

In [0]:
train_df = pd.read_csv("/content/image-classifiction-framework/dataset/train.csv")
def get_label(file_name):
  parts = tf.strings.split(file_name, "/")
  print(parts[-1])
  file_name = file_name.split("/")[-1].strip(".png")
  # convert the path to a list of path components
  class_id = train_df[train_df["id_code"] == file_name]["diagnosis"].iloc[0]
  # map to class dictionary
  class_name = CLASS_MAPPING_DICT[class_id]
  # The second to last is the class-directory
  return parts[-2] == CLASS_NAMES

In [0]:
def decode_img(img):
  tf.image.decode_png
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_png(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [0]:
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [101]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)

Tensor("strided_slice:0", shape=(), dtype=string)


AttributeError: ignored